In [ ]:
!pip install streamlit pyngrok tensorflow pillow joblib torch transformers pytorch-tabnet --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 113.1 MB/s eta 0:00:00


In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("2zELGoE2ygB2rvU3KVHSB5A7qqh_71xvBX1vyBpa2MvGgTdSN")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%writefile app.py
import streamlit as st
import numpy as np
import joblib
import torch
import json
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# ===================== BASE PATH =====================
BASE_PATH = "/content/drive/MyDrive/Klasifikasi_Kategori_Judul_Berita/models"

# ===================== PAGE CONFIG =====================
st.set_page_config(
    page_title="Dashboard Klasifikasi Kategori Judul Berita",
    layout="wide",
    initial_sidebar_state="expanded"
)

st.title("📰 Dashboard Klasifikasi Kategori Judul Berita")
st.write("Prediksi menggunakan **LSTM**, **DistilBERT**, dan **IndoBERT**")

# ===================== CACHE MODELS =====================
@st.cache_resource
def load_lstm_model():
    tf.keras.backend.clear_session()
    model = load_model(f"{BASE_PATH}/lstm/model_lstm.h5", compile=False)
    model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
                  loss="categorical_crossentropy", metrics=["accuracy"])
    with open(f"{BASE_PATH}/lstm/tokenizer_lstm.json", "r") as f:
        tokenizer_json = f.read()
        tokenizer = tf.keras.preprocessing.text.tokenizer_from_json(tokenizer_json)
    label_encoder = joblib.load(f"{BASE_PATH}/lstm/label_encoder_lstm.pkl")
    return model, tokenizer, label_encoder

@st.cache_resource
def load_distilbert_model():
    model_path = f"{BASE_PATH}/distilbert"
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForSequenceClassification.from_pretrained(model_path)
    label_encoder = joblib.load(f"{model_path}/label_encoder.pkl")
    return tokenizer, model, label_encoder

@st.cache_resource
def load_indobert_model():
    model_path = f"{BASE_PATH}/indobert"
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForSequenceClassification.from_pretrained(model_path)
    label_encoder = joblib.load(f"{model_path}/label_encoder.pkl")
    return tokenizer, model, label_encoder

# ===================== LOAD MODELS =====================
lstm_model, lstm_tokenizer, lstm_label_encoder = load_lstm_model()
distil_tokenizer, distil_model, distil_label_encoder = load_distilbert_model()
indo_tokenizer, indo_model, indo_label_encoder = load_indobert_model()

# ===================== TABS =====================
tab_lstm, tab_distil, tab_indo = st.tabs(["🟢 LSTM", "🟡 DistilBERT", "🔵 IndoBERT"])

# ===================== LSTM =====================
with tab_lstm:
    st.header("🟢 Prediksi dengan LSTM")
    text_input_lstm = st.text_area("Masukkan judul berita:", key="lstm_text", height=150)
    if st.button("Prediksi LSTM", key="btn_lstm"):
        if text_input_lstm.strip() == "":
            st.warning("Teks tidak boleh kosong")
        else:
            seq = lstm_tokenizer.texts_to_sequences([text_input_lstm])
            pad_seq = pad_sequences(seq, maxlen=100, padding="post", truncating="post")
            pred = lstm_model.predict(pad_seq)
            label = lstm_label_encoder.inverse_transform([np.argmax(pred)])[0]
            confidence = np.max(pred) * 100
            st.success(f"Hasil Prediksi: **{label}**")
            st.info(f"Confidence: **{confidence:.2f}%**")

# ===================== DistilBERT =====================
with tab_distil:
    st.header("🟡 Prediksi dengan DistilBERT")
    text_input_distil = st.text_area("Masukkan judul berita:", key="distil_text", height=150)
    if st.button("Prediksi DistilBERT", key="btn_distil"):
        if text_input_distil.strip() == "":
            st.warning("Teks tidak boleh kosong")
        else:
            inputs = distil_tokenizer(text_input_distil, return_tensors="pt", truncation=True, padding=True, max_length=256)
            with torch.no_grad():
                outputs = distil_model(**inputs)
            probs = torch.softmax(outputs.logits, dim=1)
            pred_idx = torch.argmax(probs, dim=1).item()
            label = distil_label_encoder.inverse_transform([pred_idx])[0]
            confidence = probs[0][pred_idx].item() * 100
            st.success(f"Hasil Prediksi: **{label}**")
            st.info(f"Confidence: **{confidence:.2f}%**")

# ===================== IndoBERT =====================
with tab_indo:
    st.header("🔵 Prediksi dengan IndoBERT")
    text_input_indo = st.text_area("Masukkan judul berita:", key="indo_text", height=150)
    if st.button("Prediksi IndoBERT", key="btn_indo"):
        if text_input_indo.strip() == "":
            st.warning("Teks tidak boleh kosong")
        else:
            inputs = indo_tokenizer(text_input_indo, return_tensors="pt", truncation=True, padding=True, max_length=256)
            with torch.no_grad():
                outputs = indo_model(**inputs)
            probs = torch.softmax(outputs.logits, dim=1)
            pred_idx = torch.argmax(probs, dim=1).item()
            label = indo_label_encoder.inverse_transform([pred_idx])[0]
            confidence = probs[0][pred_idx].item() * 100
            st.success(f"Hasil Prediksi: **{label}**")
            st.info(f"Confidence: **{confidence:.2f}%**")

Overwriting app.py


In [ ]:
!streamlit run app.py --server.port 8501 --server.address 0.0.0.0 &>/content/logs.txt &

In [ ]:
from pyngrok import ngrok

public_url = ngrok.connect(8501)
print("🔗 Streamlit URL:", public_url)

🔗 Streamlit URL: NgrokTunnel: "https://e84b4dfb9e0b.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
!pkill ngrok